## Run ASHLAR to assemble and align the tiles.

In [1]:
import os
import json
import subprocess

In [2]:
# define input and output pathways
input_dir = r''

dict_path = r''

output_dir = r''

In [ ]:
# use a dictionary to map round and channel to the corresponding channel name
# this is used to create the output file name

# read dictionary from json file

signal_dict = json.load(open(dict_path))

# or specify manually
# '0_ 0' corresponds to round 0, channel 0 (space is added to match the format of the input file names)
#signal_dict = {} # example {'0_ 0':'DAPI1','0_ 1':'RB','0_ 2':'p21','0_ 3':'pH3','1_ 0':'DAPI2','1_ 1':'Cdc25c','1_ 2':'pCDK1','1_ 3':'Fib'}

In [ ]:
# specify wells to process
well_list = [''] # ex. ['A1']

# or run for all wells
#well_list = list(set([x.split('_')[0] for x in os.listdir(input_dir) if 'tif' in x]))

print(len(well_list))
well_list

In [ ]:
# specify rounds to process
round_list = [] 

# or run for all rounds
round_list = sorted(list(set([x.split('_')[2] for x in os.listdir(input_dir) if 'tif' in x])))

print(len(round_list))
round_list

In [17]:
for label in well_list:
    
    # Construct the file series arguments for all rounds
    fileseries_args = [f"fileseries|{input_dir}|pattern={label}_r_{round_num}_ch_{{channel:2}}_{{series:2}}.tif|overlap=0.105|width=9|height=9|layout=snake|pixel_size=0.325" for round_num in round_list]

    # Create the full command as a list of arguments
    command = [
        'ashlar',
        *fileseries_args,
        '--output', f"{output_dir}\\{label}_ashlar_{{cycle:2}}_{{channel:2}}.tif",
        '--maximum-shift', '200',
        '--align-channel', '0'
    ]

    # Execute the command using subprocess
    subprocess.run(command)

### Rename the output files to include the channel name

In [ ]:
im_list = os.listdir(output_dir)

for im_name in im_list:
    signal_name = signal_dict[('_').join(im_name.split('_')[-2:])[1:-4]]
    old_path = os.path.join(output_dir,im_name)  
    new_name = im_name.split('_')[:2]
    new_name.append(signal_name+'.tif')  
    new_path = os.path.join(output_dir,'_'.join(new_name))
    print(new_path)
    os.rename(old_path, new_path)